## Feature vectors

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.ndimage import uniform_filter1d
import re
import numpy as np
import os
from ultralytics import YOLO
import cv2

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
def get_data_files(folder_path):
    video_files = []

    for root, dirs, files in os.walk(folder_path):
        subfolder_list = root.split(os.path.sep)
        v_nv = subfolder_list[-2]
        c1_c2 = subfolder_list[-1]

        video_files.extend([(v_nv, c1_c2, file) for file in files if file.lower().endswith(('.mp4', '.jpg'))])

    return video_files

In [ ]:
keyframes_dataset_path = '/content/drive/MyDrive/AIRTLab/'

image_files_list = get_data_files(keyframes_dataset_path)

In [ ]:
columns = ('filename','keypoints', 'label', 'set_split')

data = []

model = YOLO("yolov8n-pose.pt")

for set_split, label, image_file in image_files_list:

  results  = []

  file_path = f'{keyframes_dataset_path}/{set_split}/{label}/{image_file}'

  estimation_result = model(file_path)

  for r in estimation_result:
    results.append(r.keypoints.xy.cpu().numpy())

  data.append((image_file, results, label, set_split))


df = pd.DataFrame(data, columns=columns)

Streaming output truncated to the last 5000 lines.
image 1/1 /content/drive/MyDrive/AIRTLab/train/violent/v61f112.jpg: 384x640 4 persons, 186.0ms
Speed: 3.1ms preprocess, 186.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/drive/MyDrive/AIRTLab/train/violent/v61f109.jpg: 384x640 4 persons, 172.9ms
Speed: 3.2ms preprocess, 172.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/drive/MyDrive/AIRTLab/train/violent/v61f114.jpg: 384x640 4 persons, 170.4ms
Speed: 4.5ms preprocess, 170.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/drive/MyDrive/AIRTLab/train/violent/v61f105.jpg: 384x640 3 persons, 162.6ms
Speed: 3.2ms preprocess, 162.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/drive/MyDrive/AIRTLab/train/violent/v61f83.jpg: 384x640 4 persons, 168.3ms
Speed: 3.1ms preprocess, 168.3ms inference, 1.2ms postprocess per image at shap

In [ ]:
df.head()
df.to_pickle('keyframes_keypoints.pkl')

In [ ]:
df.head()

,filename,keypoints,label,set_split
0,v37f2.jpg,"[[[[ 1504.3 260.41], [ 1521.8 ...",non-violent,test
1,v37f4.jpg,"[[[[ 735.88 116.31], [ 746.55 ...",non-violent,test
2,v37f1.jpg,"[[[[ 1510.4 265.2], [ 1526.1 ...",non-violent,test
3,v37f8.jpg,"[[[[ 770.74 122.25], [ 781.85 ...",non-violent,test
4,v37f26.jpg,"[[[[ 1113.5 347.87], [ 1126.2 ...",non-violent,test


In [ ]:
def keypoint_statistics_features(keypoints_sequence):
    if keypoints_sequence.shape[0] < 1:
        return None  # Not enough frames

    smoothed_sequence = uniform_filter1d(keypoints_sequence, size=3, axis=0, mode='nearest')

    stats = [
        np.min(smoothed_sequence, axis=0),
        np.max(smoothed_sequence, axis=0),
        np.mean(smoothed_sequence, axis=0),
        np.std(smoothed_sequence, axis=0),
        np.median(smoothed_sequence, axis=0),
        np.max(smoothed_sequence, axis=0) - np.min(smoothed_sequence, axis=0),
        np.percentile(smoothed_sequence, 25, axis=0),
        np.percentile(smoothed_sequence, 75, axis=0),
    ]


    features = [stat.flatten() for stat in stats]
    return np.concatenate(features)

def aggregate_video_features2(video_person_skeletons, pooling='mean'):
    person_features = []

    for keypoints_sequence in video_person_skeletons:
        feats = keypoint_statistics_features(np.array(keypoints_sequence))
        if feats is not None:
            person_features.append(feats)

    if not person_features:
        return np.zeros(204)

    person_features = np.array(person_features)

    if pooling == 'mean':
        return np.mean(person_features, axis=0)
    elif pooling == 'max':
        return np.max(person_features, axis=0)
    else:
        raise ValueError("Pooling must be 'mean' or 'max'")

In [ ]:
df = pd.read_pickle('keyframes_keypoints.pkl')
df.head()

,filename,keypoints,label,set_split
0,v37f2.jpg,"[[[[ 1504.3 260.41], [ 1521.8 ...",non-violent,test
1,v37f4.jpg,"[[[[ 735.88 116.31], [ 746.55 ...",non-violent,test
2,v37f1.jpg,"[[[[ 1510.4 265.2], [ 1526.1 ...",non-violent,test
3,v37f8.jpg,"[[[[ 770.74 122.25], [ 781.85 ...",non-violent,test
4,v37f26.jpg,"[[[[ 1113.5 347.87], [ 1126.2 ...",non-violent,test


In [ ]:
keypoints_list = np.array(df['keypoints'].copy())
video_features = []
for video in keypoints_list:
    # `video` is a dict: {person_id: [skeletons_per_frame]}
    fvec = aggregate_video_features2(video, pooling='max')
    video_features.append(fvec)

In [ ]:
df['feature_vectors'] = video_features
df.head()

,filename,keypoints,label,set_split,feature_vectors
0,v37f2.jpg,"[[[[ 1504.3 260.41], [ 1521.8 ...",non-violent,test,"[468.48264, 206.82518, 468.3542, 200.43353, 45..."
1,v37f4.jpg,"[[[[ 735.88 116.31], [ 746.55 ...",non-violent,test,"[728.47546, 165.11365, 731.4533, 159.02248, 72..."
2,v37f1.jpg,"[[[[ 1510.4 265.2], [ 1526.1 ...",non-violent,test,"[592.66693, 224.14517, 598.10016, 212.28174, 5..."
3,v37f8.jpg,"[[[[ 770.74 122.25], [ 781.85 ...",non-violent,test,"[753.60345, 177.83728, 754.611, 169.4323, 747...."
4,v37f26.jpg,"[[[[ 1113.5 347.87], [ 1126.2 ...",non-violent,test,"[820.91174, 302.14297, 821.9999, 294.54153, 81..."


In [ ]:
df.feature_vectors[0].shape

(272,)

## Fusion Model Training

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.optimizers import RMSprop
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATASET_TEST_PATH = '/content/drive/MyDrive/AIRTLab/test/'

DATASET_TRAIN_PATH = '/content/drive/MyDrive/AIRTLab/train/'

TARGET_SIZE = (120,160)

DESIRED_ACCURACY = 0.995

In [ ]:
def violence_test_dataset():
    return  image_dataset_from_directory(DATASET_TEST_PATH,
                                         batch_size=32,
                                         label_mode='categorical',
                                         image_size=TARGET_SIZE)

def violence_train_dataset():
    return  image_dataset_from_directory(DATASET_TRAIN_PATH,
                                         batch_size=32,
                                         label_mode='categorical',
                                         image_size=TARGET_SIZE)

In [ ]:
def build_fusion_model(input_shape_video=(120, 160, 3), feature_dim=272, num_classes=2):
    # ----- CNN Branch -----
    mobilenet_base = tf.keras.applications.MobileNetV2(
        input_shape=input_shape_video,
        include_top=False,
        weights='imagenet'
    )
    mobilenet_base.trainable = False

    video_input = layers.Input(shape=input_shape_video, name="video_input")
    x1 = mobilenet_base(video_input)
    x1 = layers.GlobalAveragePooling2D()(x1)

    # ----- MLP Branch -----
    keypoint_input = layers.Input(shape=(feature_dim,), name="keypoint_input")
    x2 = layers.Dense(128, activation="relu")(keypoint_input)
    x2 = layers.Dense(64, activation="relu")(x2)

    # ----- Fusion -----
    combined = layers.Concatenate()([x1, x2])
    x = layers.Dense(64, activation="relu")(combined)
    output = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs=[video_input, keypoint_input], outputs=output)

    model.compile(optimizer=RMSprop(learning_rate=0.0001),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"]
    )
    return model

In [ ]:
fusion_model = build_fusion_model()

fusion_model.summary()

/tmp/ipython-input-2085389589.py:3: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  mobilenet_base = tf.keras.applications.MobileNetV2(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ video_input         │ (None, 120, 160,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ keypoint_input      │ (None, 272)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mobilenetv2_1.00_2… │ (None, 4, 5,      │  2,257,984 │ video_input[0][0] │
│ (Functional)        │ 1280)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 128)       │     34,944 │ keypoint_input[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1280)      │          0 │ mobilenetv2_1.00… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 64)        │      8,256 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 1344)      │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 64)        │     86,080 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 2)         │        130 │ dense_6[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,387,394 (9.11 MB)

 Trainable params: 129,410 (505.51 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
def extract_labels(ds):
    labels = []
    for _, y in ds:
        labels.append(y.numpy())
    return np.concatenate(labels)

y_train = extract_labels(train_img_ds)
y_val   = extract_labels(val_img_ds)


def extract_images(ds):
    imgs = []
    for x, _ in ds:
        imgs.append(x.numpy())
    return np.concatenate(imgs)

x_train_img = extract_images(train_img_ds)
x_val_img   = extract_images(val_img_ds)

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(
    ((x_train_img, train_keypoints), y_train)
).batch(32).shuffle(1000)

val_ds = tf.data.Dataset.from_tensor_slices(
    ((x_val_img, val_keypoints), y_val)
).batch(32)

In [ ]:
df = pd.read_pickle('keyframes_keypoints.pkl')
df.head()

,filename,keypoints,label,set_split
0,v37f2.jpg,"[[[[1504.3322 260.4124], [1521.8273 255.220...",non-violent,test
1,v37f4.jpg,"[[[[735.87787 116.309715], [746.5505 110.004...",non-violent,test
2,v37f1.jpg,"[[[[1510.4242 265.19766], [1526.0929 258.19...",non-violent,test
3,v37f8.jpg,"[[[[770.736 122.24625], [781.85016 114.03099...",non-violent,test
4,v37f26.jpg,"[[[[1113.5089 347.87 ], [1126.2001 341.211 ...",non-violent,test


In [ ]:
keypoints_train = df[df.set_split == "train"]
keypoints_test = df[df.set_split == "test"]

In [ ]:
def reorder_df_by_paths(df, file_paths):
    df["path"] = "/content/drive/MyDrive/AIRTLab/" + df["set_split"] + "/" + df["label"] + "/" + df["filename"]

    order_df = pd.DataFrame({"path": file_paths})
    order_df["order"] = range(len(order_df))

    merged = df.merge(order_df, on="path", how="inner")
    merged = merged.sort_values("order").drop(columns=["path", "order"])

    return merged.reset_index(drop=True)

In [ ]:
file_paths = treino.file_paths
df = pd.read_pickle('keyframes_keypoints.pkl')
new_df = reorder_df_by_paths(df, file_paths)

In [ ]:
from sklearn.model_selection import train_test_split
from scipy.ndimage import uniform_filter1d

def keypoint_statistics_features(keypoints_sequence):
    if keypoints_sequence.shape[0] < 1:
        return None  # Not enough frames

    smoothed_sequence = uniform_filter1d(keypoints_sequence, size=3, axis=0, mode='nearest')

    stats = [
        np.min(smoothed_sequence, axis=0),
        np.max(smoothed_sequence, axis=0),
        np.mean(smoothed_sequence, axis=0),
        np.std(smoothed_sequence, axis=0),
        np.median(smoothed_sequence, axis=0),
        np.max(smoothed_sequence, axis=0) - np.min(smoothed_sequence, axis=0),
        np.percentile(smoothed_sequence, 25, axis=0),
        np.percentile(smoothed_sequence, 75, axis=0),
    ]


    features = [stat.flatten() for stat in stats]
    return np.concatenate(features)

def aggregate_video_features(video_person_skeletons, pooling='mean'):
    person_features = []

    for keypoints_sequence in video_person_skeletons:
        feats = keypoint_statistics_features(np.array(keypoints_sequence))
        if feats is not None:
            person_features.append(feats)

    if not person_features:
        return np.zeros(204)

    person_features = np.array(person_features)

    if pooling == 'mean':
        return np.mean(person_features, axis=0)
    elif pooling == 'max':
        return np.max(person_features, axis=0)
    else:
        raise ValueError("Pooling must be 'mean' or 'max'")

In [ ]:
keypoints_list = np.array(new_df['keypoints'].copy())
video_features = []
for video in keypoints_list:
    # `video` is a dict: {person_id: [skeletons_per_frame]}
    fvec = aggregate_video_features(video, pooling='max')
    video_features.append(fvec)

In [ ]:
new_df['feature_vectors'] = video_features

,filename,keypoints,label,set_split,feature_vectors
0,v58f122.jpg,"[[[[691.67084 749.1396 ], [689.95734 745.5522 ...",violent,train,"[696.0596, 173.64444, 698.8789, 163.12158, 700..."
1,v4f24.jpg,"[[[[1421.1746 226.76962], [1432.3367 218.3...",non-violent,train,"[1270.2258, 248.55927, 1274.2936, 240.27287, 1..."
2,v35f119.jpg,"[[[[1391.4457 361.47284], [1395.4491 351.2...",non-violent,train,"[1298.3293, 349.26022, 1298.6206, 338.50943, 1..."
3,v9f92.jpg,"[[[[914.76764 323.44522], [917.78033 311.57108...",non-violent,train,"[901.1548, 147.03871, 909.8066, 135.22134, 896..."
4,v86f107.jpg,"[[[[1446.7942 494.67288], [1461.2084 485.6...",violent,train,"[849.5586, 403.15393, 857.6884, 394.13953, 841..."
